# Data preperation for Dashboards - State_Evaluation project

<p> <h3> Summary </h3> <ol> 
<li> I have taken multiple files and an URL and read them into seperate dataframes </li>
<li> I have cleaned the data as required. From each frame, I have reatined State and the required parameter/s </li>
<li> Finally I am merging the dataframes by state and writing the merged dataframe into csv file </li> </ol> </p>

In [1]:
#Importing all the required libraries

import pandas as pd
import numpy as np
import math
import xlrd

In [2]:
#Making a list of all the states in United States. This list will be reused across multiple datasets as a filter

states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
         'Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho', 
         'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
         'Maine' 'Maryland','Massachusetts','Michigan','Minnesota',
         'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
         'New Hampshire','New Jersey','New Mexico','New York',
         'North Carolina','North Dakota','Ohio',    
         'Oklahoma','Oregon','Pennsylvania','Rhode Island',
         'South  Carolina','South Dakota','Tennessee','Texas','Utah',
         'Vermont','Virginia','Washington','West Virginia',
         'Wisconsin','Wyoming']

### Business parameters preperation

In [3]:
state_rank_url = "http://www.cnbc.com/2016/07/12/americas-top-states-for-business-2016-the-list-and-ranking.html"
state_ranking_data = pd.read_html(state_rank_url, header = 0)[0]
state_ranking_data = pd.read_html(state_rank_url, header = 0)[0]

In [4]:
state_ranking_data.head(5)

,Overall,State,Workforce,Cost of Doing Business,Infra-structure,Economy,Quality of Life,Technology and Innovation,Education,Business Friendliness,Cost of Living,Access to Capital
0,1,Utah,12,19,13,3,13,16,23,10,22,24
1,2,Texas,8,23,2,1,37,10,40,18,21,7
2,3,Colorado,1,37,29,2,12,8,25,9,32,8
3,4,Minnesota,15,35,5,17,2,9,2,27,27,21
4,5,North Carolina,9,18,30,11,30,7,28,15,18,2


### Education levels data

In [5]:
# The data is obtained at county level. Hence aggregating at a state level. We need the percentage of high school 
# graduates and above. Hence subtracting Percent of adults with less than a high school diploma from 100.

education_data = pd.read_excel('Education.xls', skiprows=4)
education_data = education_data[education_data['Area name'].isin(states)]
education_data['% of adults with high school diploma'] = 100 - education_data['Percent of adults with less than a high school diploma, 2011-2015']
education_data = education_data[['Area name','% of adults with high school diploma']]
education_data = education_data.rename(columns={'Area name': 'State'})

In [6]:
education_data.head(5)

,State,% of adults with high school diploma
1,Alabama,84.3
69,Alaska,92.1
106,Arizona,86.0
122,Arkansas,84.8
198,California,81.8


### Unemplyment rates data

In [9]:
# This data is obtained both at county level and state level. Retaining only state level data

unemp_raw = pd.read_excel('Unemployment.xls', skiprows = 7)
unemp_raw = unemp_raw[unemp_raw['Area_name'].isin(states)]
unemp_data = unemp_raw[['Area_name','Unemployment_rate_2015']]
unemp_data = unemp_data.rename(columns={'Area_name': 'State', 'Unemployment_rate_2015': 'Unemplyment_rate'})

In [10]:
unemp_data.head()

,State,Unemplyment_rate
0,Alabama,6.1
68,Alaska,6.5
101,Arizona,6.1
117,Arkansas,5.2
193,California,6.2


### Crime data 

In [11]:
# This data is obtained both at county level and state level. Retaining only state level data and differentiating 
# crimes into 2 broad categories - Felony and Robbery

crime_raw = pd.read_excel('US Crime Data.xlsx',converters={'Location Ref Code':str})
crime_raw.drop(['Location','Location RegionId','Indicator','Location UNITED STATES','Location US','Scale','Units'], axis=1, inplace=True)
crime_raw = crime_raw[crime_raw['Location Name'].isin(states)]
crime_raw = crime_raw[crime_raw['Indicator Name'].isin(['Number of Violent Crimes Known to Police','Number of Property Crimes Known to Police'])]
crime_raw = crime_raw[crime_raw['Location Name'].isin(states)]
crime_raw = crime_raw[crime_raw['Indicator Name'].isin(['Number of Violent Crimes Known to Police','Number of Property Crimes Known to Police'])]
crime_raw.dropna(subset = ['Location Ref Code'], inplace = True)
crime_raw['Felony'] = crime_raw[crime_raw['Indicator Name'].isin(['Number of Violent Crimes Known to Police'])]['2015']
crime_raw['Robbery'] = crime_raw[crime_raw['Indicator Name'].isin(['Number of Property Crimes Known to Police'])]['2015']
crime_data = crime_raw.groupby(by = 'Location Name')['Felony','Robbery'].sum().reset_index()
crime_data = crime_data.rename(columns={'Location Name': 'State'})

In [12]:
crime_data.head()

,State,Felony,Robbery
0,Alabama,3954.0,25447.0
1,Arizona,3098.0,23053.0
2,Arkansas,3352.0,18035.0
3,California,24994.0,105500.0
4,Colorado,2469.0,17928.0


### Population estimates

In [16]:
pop_estimate = pd.read_excel('PopulationEstimates.xls', skiprows=2)
pop_estimate = pop_estimate[pop_estimate['Area_Name'].isin(states)]
pop_estimate = pop_estimate[['Area_Name','POP_ESTIMATE_2015']]
pop_estimate = pop_estimate.rename(columns={'Area_Name': 'State', 'POP_ESTIMATE_2015': 'Population_Estimate'})

In [17]:
pop_estimate.head(5)

,State,Population_Estimate
1,Alabama,4853875
69,Alaska,737709
99,Arizona,6817565
115,Arkansas,2977853
191,California,38993940


### Merging all the dataframes by state and writing the merged dataframe into csv file

In [19]:
df_combine = [state_ranking_data, pop_estimate, unemp_data, education_data]
tableau_data = reduce(lambda left,right: pd.merge(left,right,on='State'), df_combine)
tableau_data = pd.merge(tableau_data, crime_data, how='left', on=['State'])
tableau_data.head(5)

,Overall,State,Workforce,Cost of Doing Business,Infra-structure,Economy,Quality of Life,Technology and Innovation,Education,Business Friendliness,Cost of Living,Access to Capital,Population_Estimate,Unemplyment_rate,% of adults with high school diploma,Felony,Robbery
0,1,Utah,12,19,13,3,13,16,23,10,22,24,2990632,3.5,91.2,1270.0,16333.0
1,2,Texas,8,23,2,1,37,10,40,18,21,7,27429639,4.5,81.9,21060.0,122184.0
2,3,Colorado,1,37,29,2,12,8,25,9,32,8,5448819,3.9,90.7,2469.0,17928.0
3,4,Minnesota,15,35,5,17,2,9,2,27,27,21,5482435,3.7,92.4,1434.0,16336.0
4,5,North Carolina,9,18,30,11,30,7,28,15,18,2,10035186,5.7,85.8,4683.0,50396.0


In [20]:
# Writing the cleaned dataframe into csv file with no index
tableau_data.to_csv('State_Evaluation.csv', sep=';', index=False)